In [3]:
%pip install opencv-python tensorflow


Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input

model_path = os.path.join("../..", "model", "waste_classifier_v1.keras")
# 1) טען את המודל ששמרת
model = load_model(model_path)

# 2) הגדר רשימת שמות הקטגוריות בהתאמה לסדר שהמודל מחזיר
class_names = ["blue", "brown", "green", "orange", "yellow"]

# 3) הגדר את גודל התמונה שהמודל שלך מצפה לו (למשל 224x224 ל-EfficientNetB3)
IMG_SIZE = (224, 224)

# 4) פתח את המצלמה
cap = cv2.VideoCapture(0)  # 0 = מצלמת ברירת המחדל

# משתנה שיעזור לנו לבדוק כל כמה פריימים נעשה חיזוי
frame_count = 0
prediction_text = ""  # נשמור פה את התחזית הנוכחית

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # כל 10 פריימים נריץ חיזוי
    if frame_count % 2 == 0:
        # עיבוד תמונה מקדימה
        # 1. שנה גודל
        resized = cv2.resize(frame, IMG_SIZE)
        resized = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
        resized = preprocess_input(resized.astype('float32'))
        input_tensor = np.expand_dims(resized, axis=0)
        preds = model.predict(input_tensor)
        print("Preds:", preds)
        class_idx = np.argmax(preds[0])
        confidence = preds[0][class_idx]

        # טקסט של התחזית
        prediction_text = f"{class_names[class_idx]} ({confidence:.2f})"

    # הוסף טקסט לפריים (ללא קשר ל־10 פריימים - יישאר עד הבא)
    cv2.putText(
        frame, 
        prediction_text, 
        (20, 40),  # מיקום הטקסט על המסך
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,  # גודל טקסט
        (0, 255, 0),  # צבע הטקסט (B,G,R)
        2,  # עובי הקו
        cv2.LINE_AA
    )

    # הצג את הווידאו
    cv2.imshow("Camera Inference", frame)

    # אם לוחצים על 'q' - צא
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 